In [1]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
async def get_expiry(f_today, index):

    if index == 'bnf':    
        if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
            f_expiry = dt.date(2024, 1, 25)
        elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
            f_expiry = dt.date(2024, 1, 31)
        elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
            f_expiry = dt.date(2024, 2, 29)
        elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
            f_expiry = dt.date(2024, 2, 27)
        elif f_today < dt.date(2023, 9, 1):
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
        elif f_today >= dt.date(2023, 9, 1):
            if f_today.day < 24:
                days_to_wednesday = (2 - f_today.weekday()) % 7
                nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
                f_expiry = nearest_wednesday
                if nse.valid_days(
                    start_date=nearest_wednesday, end_date=nearest_wednesday
                ).empty:
                    f_expiry = nearest_wednesday - dt.timedelta(days=1)
            else:
                days_to_thursday = (3 - f_today.weekday()) % 7
                nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
                f_expiry = nearest_thursday
                if nse.valid_days(
                    start_date=nearest_thursday, end_date=nearest_thursday
                ).empty:
                    f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'nifty':
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'finnifty' or index == 'fnf':
        days_to_thursday = (1 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'midcpnifty' or index == 'midcp':
        days_to_thursday = (0 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"


def get_option_contract_name2(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [687]:
bnf_1min = pd.read_csv("../data/bn_1h_tv.csv")
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["time"])
bnf_1min = bnf_1min[bnf_1min["datetime"].dt.year >= 2017]

In [688]:
bnf_1min.tail()

,time,open,high,low,close,MA,MA.1,EMA,datetime
20777,2025-01-20T11:15:00+05:30,49052.4500,49374.8500,49051.0500,49311.3000,48785.1231,49008.4423,48941.2075,2025-01-20 11:15:00+05:30
20778,2025-01-20T12:15:00+05:30,49315.9000,49562.0500,49291.7500,49541.8500,48812.1750,49029.7769,49112.8197,2025-01-20 12:15:00+05:30
20779,2025-01-20T13:15:00+05:30,49545.3000,49650.6000,49399.1000,49463.9500,48838.3981,49058.2327,49213.1426,2025-01-20 13:15:00+05:30
20780,2025-01-20T14:15:00+05:30,49467.3500,49489.6000,49359.7000,49386.9500,48865.5788,49076.5442,49262.8019,2025-01-20 14:15:00+05:30
20781,2025-01-20T15:15:00+05:30,49389.3500,49389.3500,49295.9000,49334.5500,48882.0365,49088.3019,49283.3013,2025-01-20 15:15:00+05:30


In [689]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [690]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"])
list_of_traded_dates = set(bnf_1min["datetime"].dt.date)
list_of_traded_dates

{datetime.date(2017, 6, 6),
 datetime.date(2020, 10, 23),
 datetime.date(2022, 6, 29),
 datetime.date(2024, 12, 27),
 datetime.date(2022, 1, 10),
 datetime.date(2024, 6, 21),
 datetime.date(2019, 1, 9),
 datetime.date(2018, 10, 12),
 datetime.date(2020, 6, 4),
 datetime.date(2023, 2, 6),
 datetime.date(2024, 7, 26),
 datetime.date(2024, 8, 29),
 datetime.date(2022, 3, 14),
 datetime.date(2024, 10, 30),
 datetime.date(2017, 8, 17),
 datetime.date(2024, 2, 2),
 datetime.date(2017, 10, 23),
 datetime.date(2022, 6, 20),
 datetime.date(2022, 7, 18),
 datetime.date(2022, 12, 20),
 datetime.date(2019, 3, 28),
 datetime.date(2020, 8, 12),
 datetime.date(2023, 12, 22),
 datetime.date(2019, 5, 10),
 datetime.date(2022, 2, 8),
 datetime.date(2018, 2, 27),
 datetime.date(2023, 5, 12),
 datetime.date(2020, 7, 16),
 datetime.date(2017, 2, 17),
 datetime.date(2024, 5, 13),
 datetime.date(2023, 5, 29),
 datetime.date(2023, 3, 15),
 datetime.date(2024, 1, 30),
 datetime.date(2019, 10, 29),
 datetime.da

In [691]:
def rename_ohlc_columns(df: pl.DataFrame) -> pl.DataFrame:

    column_mapping = {"o": "open", "h": "high", "l": "low", "c": "close", "v": "volume"}
    df = df.rename(column_mapping)

    return df

In [692]:
PORTFOLIO_VALUE = 10_00_000 # 10 Lacs
# RPT_PCT = 0.01 # 1% RPT
SLIPPAGE_ = 0.0001
LEVERAGE_ = 5

In [693]:
def calculate_signals(df, high_window1, high_window2, low_window, ema_length):
   
    # Calculate the highest high for the previous `high_window1` candles
    df['Prev_High_20'] = df['high'].rolling(window=high_window1).max()
    
    # Calculate the highest high for the range from `i-21` to `i-60`
    df['Prev_High_40'] = df['high'].shift(high_window1).rolling(window=high_window2).max()
    
    # Calculate the lowest low for the last `low_window` candles
    df['Prev_Low_6'] = df['low'].rolling(window=low_window).min()

    df['EMA_100'] = df['close'].ewm(span=ema_length, adjust=False).mean()

    # Sell signal conditions
    df['Sell_Signal'] = (
        (df['Prev_High_20'] > df['Prev_High_40']) &
        (df['low'] < df['Prev_Low_6'].shift(1))
    )

    # Drop intermediate columns if not needed
    df.drop(columns=['Prev_High_20', 'Prev_High_40', 'Prev_Low_6'], inplace=True)

    return df

In [737]:
def backtest3(df):
    # print(df.to_string())
    # return
    df.reset_index(drop=True, inplace=True)
    # Variables for short trades
    short_position = 0  # 0 = no position, 1 = short
    short_entry_price = 0
    short_entry_date = None
    short_trades = []
    tradebook = pd.DataFrame()
    tradebook_short = pd.DataFrame()
    # tradebook_short = pd.DataFrame()
    short_trailing_stop = None
    # Variables for short trades
    # short_position = 0  # 0 = no position, -1 = short
    # short_entry_price = 0
    # short_entry_date = None
    # short_trades = []
    # short_trailing_stop = None

    can_add_short = False
    # can_add_short = False

    trade_number = 0
    lock_initial_sl = False
    first_sl = 0

    lock_initial_sl_2 = False
    first_sl_2 = 0

    pyramid_number = 0
    can_pyramid = True
    pyramid_high = 0

    for i in range(1, len(df)):
        # Entry signals for short trades
        short_trailing_stop = df.loc[i, 'EMA_100']
        # short_trailing_stop = df.loc[i, 'EMA_20']
        
        if df.loc[i, 'Sell_Signal'] and short_position == 0:
            short_position = 1
            short_entry_price = df.loc[i, 'close']
            short_entry_date = df.loc[i, 'datetime']
            short_trailing_stop = df.loc[i, 'EMA_100']
            # short_initial_sl = df.loc[i, 'X_High']
            short_initial_sl = short_entry_price * 1.025

            tradebook_short = pd.concat([tradebook_short, pd.DataFrame([{
                'Trade No.': trade_number,
                'Entry_Date': df.loc[i, 'datetime'],
                'Exit_Date': None,
                'Trade_Type': 'short',
                'Entry_Price': df.loc[i, 'close'],
                # 'Previous MA Value': df.loc[i-1, 'X_Low'],
                'Initial SL': short_initial_sl,
                'Exit_Price': None,
                # 'Profit/Loss': None  # Filled when exited
            }])], ignore_index=True)

            if not lock_initial_sl:
                first_sl = short_initial_sl
                lock_initial_sl = True

            can_pyramid = True
            pyramid_number = 0
            pyramid_high = 0
            continue
                
        # Exit or add to short position
        if short_position == 1:

            
            # y_days_low = df['low'].rolling(y_days).min().iloc[i-1]
            # y_days_high = df['high'].rolling(y_days).max().iloc[i-1]
            
            # print(df.iloc[i]['datetime'], y_days_high, y_days_low)
            
            if df.loc[i, 'high'] >= short_initial_sl:
                if df.loc[i, 'open'] >= short_initial_sl:
                    tradebook_short.loc[
                        (tradebook_short['Trade No.'] == trade_number),
                        ['Exit_Date', 'Exit_Price', 'Exit Remark']
                    ] = [df.loc[i, 'datetime'] , df.loc[i, 'open'], 'Gap']
                    short_position = 0
                    can_add_short = False
                    trade_number += 1
                    lock_initial_sl = False
                    first_sl = 0
                    can_pyramid = False
                    pyramid_number = 0
                    pyramid_high = 0
                    # if df.loc[i, 'high'] >= long_initial_sl:
                    #     tradebook_long.loc[
                    #         (tradebook_long['Trade No.'] == trade_number),
                    #         ['Exit_Date', 'Exit_Price', 'Exit Remark']
                    #     ] = [df.loc[i, 'datetime'] , long_initial_sl, 'Initial SL Hit']
                    #     long_position = 0
                    #     can_add_long = False
                    #     trade_number += 1
                    #     lock_initial_sl = False
                    #     first_sl = 0
                    #     can_pyramid = False
                    #     pyramid_number = 0
                    #     pyramid_high = 0
                    # elif df.loc[i, 'high'] < long_initial_sl:
                    #     tradebook_long.loc[
                    #         (tradebook_long['Trade No.'] == trade_number),
                    #         ['Exit_Date', 'Exit_Price', 'Exit Remark']
                    #     ] = [df.loc[i, 'datetime'] , df.loc[i, 'close'], 'Gap Open-Close Below ISL']
                    #     long_position = 0
                    #     can_add_long = False
                    #     trade_number += 1
                    #     lock_initial_sl = False
                    #     first_sl = 0
                    #     can_pyramid = False
                    #     pyramid_number = 0
                    #     pyramid_high = 0
                else:
                    tradebook_short.loc[
                        (tradebook_short['Trade No.'] == trade_number),
                        ['Exit_Date', 'Exit_Price', 'Exit Remark']
                    ] = [df.loc[i, 'datetime'] , short_initial_sl, 'Initial SL Hit']
                    short_position = 0
                    can_add_short = False
                    trade_number += 1
                    lock_initial_sl = False
                    first_sl = 0
                    can_pyramid = False
                    pyramid_number = 0
                    pyramid_high = 0
            
            elif df.loc[i, 'close'] >= short_trailing_stop:
                tradebook_short.loc[
                    (tradebook_short['Trade No.'] == trade_number),
                    ['Exit_Date', 'Exit_Price', 'Exit Remark']
                ] = [df.loc[i, 'datetime'] , df.loc[i, 'close'] , 'Trailing SL Hit']
                short_position = 0
                can_add_short = False
                trade_number += 1
                lock_initial_sl = False
                first_sl = 0
                can_pyramid = False
                pyramid_number = 0
                pyramid_high = 0

            # elif (df.loc[i, 'low'] <= y_days_low) and not can_add_long:
            #     # Previous Y Candle Low Breached, confirmation found. Add position on high break
            #     can_add_long = True

            # if can_add_long and long_position and can_pyramid:
            #     if df.loc[i, 'high'] >= y_days_high and pyramid_number < pyr_num:
            #         tradebook_long = pd.concat([tradebook_long, pd.DataFrame([{
            #             'Trade No.': trade_number,
            #             'Entry_Date': df.loc[i, 'datetime'],
            #             'Exit_Date': None,
            #             'Trade_Type': 'Add_Long',
            #             'Entry_Price': y_days_high,
            #             'Initial SL': first_sl,
            #             'Exit_Price': None,
            #             # 'Profit/Loss': None  # Filled when exited
            #         }])], ignore_index=True)
            #         can_add_long = False
            #         pyramid_number += 1
            #         can_pyramid = True
    
    tradebook = pd.concat([tradebook_short, tradebook], ignore_index=True)
    return tradebook


In [979]:
# ema_length = 50
# x_days = 12
# # y_days = 4
# pct = 0.1
RPT = 2.5
df = bnf_1min
# df = calculate_signals(df, 50, 100, 6, 6, 0.1)
df = calculate_signals(df, 120, 600, 40, 50)
# print(df.tail().to_string())
# df1 = df[df['Buy_Signal']]
# print(df1.to_string())
tb = backtest3(df)
tb = tb.sort_values(by='Entry_Date')
# variation = f'EMA: {ema_length}, X: {x_days}, Y: {y_days}, PCT: {pct}%, RPT: {RPT}%'

/tmp/ipykernel_13340/244031479.py:133: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_short.loc[


In [980]:
import numpy as np
portfolio = 1000000
tb['PnL'] = np.where(
    tb['Trade_Type'].isin(['long']),  # For Long and Add_Long
    tb['Exit_Price'] - tb['Entry_Price'],
    np.where(
        tb['Trade_Type'].isin(['short']),  # For Short and Add_Short
        tb['Entry_Price'] - tb['Exit_Price'],
        0  # Default case if Trade_Type is something unexpected
    )
)

tb['Slippage'] = SLIPPAGE_ * (tb['Entry_Price'] + tb['Exit_Price'])
tb['PnL w cs'] = tb['PnL'] - tb['Slippage']
tb['Qty'] = abs(RPT / 100 * portfolio / (tb['Entry_Price'] - tb['Initial SL']))
# tb['Qty'] = portfolio / tb['Entry_Price']
tb['Profit_Loss INR'] = tb['Qty'] * tb['PnL w cs']
tb['ROI%'] = tb['Profit_Loss INR'] * 100 / portfolio
tb['Trade Year'] = tb['Entry_Date'].dt.year

In [981]:
tb = tb.reset_index()

In [982]:
# tb

In [983]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    
    # print(f'{overall_total_roi} , {overall_max_drawdown} , {overall_roi_dd_ratio}')
    
    return {overall_roi_dd_ratio: stats_df8}

In [984]:
tb['Entry_Date'] = pd.to_datetime(tb['Entry_Date'])
tb['Trade Year'] = tb['Entry_Date'].dt.year
# tb['ROI% w cs'] = tb['ROI%']

In [985]:
stats = generate_stats(tb, 'Short')
lol = pd.DataFrame()
for x, y in stats.items():
    lol = pd.DataFrame(y)

lol

/tmp/ipykernel_13340/2611632047.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2017,-0.4466,8,37.5000,0.7347,-0.5301,-1.6366,-0.2729,Short
2018,16.0948,3,100.0000,5.3649,NaN,0.0000,inf,Short
2019,-4.7908,9,22.2222,0.5569,-0.8435,-3.9672,-1.2076,Short
2020,7.3490,5,100.0000,1.4698,NaN,0.0000,inf,Short
2021,-0.7017,8,37.5000,2.3504,-1.5506,-6.0358,-0.1163,Short
2022,0.1949,5,20.0000,3.8142,-0.9048,-1.4882,0.1309,Short
2023,-1.6393,11,18.1818,1.1164,-0.4302,-3.1458,-0.5211,Short
2024,1.5150,10,40.0000,1.6271,-0.8322,-3.0366,0.4989,Short
Overall,17.5752,59,38.9831,2.0160,-0.7998,-6.1219,2.8709,Short


In [977]:
tb.tail(10)

,index,Trade No.,Entry_Date,Exit_Date,Trade_Type,Entry_Price,Initial SL,Exit_Price,Exit Remark,PnL,Slippage,PnL w cs,Qty,Profit_Loss INR,ROI%,Trade Year
40,40,40,2023-07-27 14:15:00+05:30,2023-08-01 09:15:00+05:30,short,45695.2500,46837.6312,45718.2000,Trailing SL Hit,-22.9500,9.1413,-32.0913,21.8841,-702.2906,-0.0702,2023
41,41,41,2023-08-02 10:15:00+05:30,2023-08-23 10:15:00+05:30,short,45096.4500,46223.8612,44119.9500,Trailing SL Hit,976.5000,8.9216,967.5784,22.1747,21455.7545,2.1456,2023
42,42,42,2023-12-21 09:15:00+05:30,2023-12-21 12:15:00+05:30,short,47231.2000,48411.9800,47564.7500,Trailing SL Hit,-333.5500,9.4796,-343.0296,21.1724,-7262.7753,-0.7263,2023
43,43,43,2024-01-03 09:15:00+05:30,2024-01-04 09:15:00+05:30,short,47577.4500,48766.8862,47973.0000,Trailing SL Hit,-395.5500,9.5550,-405.1050,21.0184,-8514.6439,-0.8515,2024
44,44,44,2024-01-08 14:15:00+05:30,2024-01-12 10:15:00+05:30,short,47454.4500,48640.8112,47706.8000,Trailing SL Hit,-252.3500,9.5161,-261.8661,21.0728,-5518.2628,-0.5518,2024
45,45,45,2024-04-15 14:15:00+05:30,2024-04-22 10:15:00+05:30,short,47770.2000,48964.4550,47822.6000,Trailing SL Hit,-52.4000,9.5593,-61.9593,20.9336,-1297.0279,-0.1297,2024
46,46,46,2024-05-07 14:15:00+05:30,2024-05-16 14:15:00+05:30,short,48305.3000,49512.9325,47973.7000,Trailing SL Hit,331.6000,9.6279,321.9721,20.7017,6665.3576,0.6665,2024
47,47,47,2024-06-04 10:15:00+05:30,2024-06-05 13:15:00+05:30,short,47714.9500,48907.8237,48659.9500,Trailing SL Hit,-945.0000,9.6375,-954.6375,20.9578,-20007.0940,-2.0007,2024
48,48,48,2024-07-10 13:15:00+05:30,2024-07-12 10:15:00+05:30,short,52120.3500,53423.3587,52649.8000,Trailing SL Hit,-529.4500,10.4770,-539.9270,19.1864,-10359.2362,-1.0359,2024
49,49,49,2024-09-30 11:15:00+05:30,2024-10-14 09:15:00+05:30,short,53103.1500,54430.7287,51617.2000,Trailing SL Hit,1485.9500,10.4720,1475.4780,18.8313,27785.1307,2.7785,2024


In [978]:
# ema_length = 55
# x_days = 28
# y_days = 7
# pct = 1.05
# RPT = 5

pct_range = [0.05]
RPT_ = 2
stats_dictionary = {}

for ema_length in range(8, 81, 4):
    for ema_length2 in range(8, 81, 4):
        for x_days in range(4, 37, 3):
            for y_days in range(4, 37, 3):
                for pct in pct_range:
                    # RPT = 5
                    variation = f'EMA1:{ema_length}, EMA2:{ema_length2} X:{x_days}, Y:{y_days}, PCT:{pct}'
                    print(variation)
                    df = bnf_1min
                    df = calculate_signals(df, ema_length, ema_length2, x_days, y_days, pct)
                    tb = backtest3(df)
                    if len(tb) > 0:
                        tb = tb.sort_values(by='Entry_Date')
                        portfolio = 10000000
                        tb['PnL'] = np.where(
                            tb['Trade_Type'].isin(['Long', 'Add_Long']),  # For Long and Add_Long
                            tb['Exit_Price'] - tb['Entry_Price'],
                            np.where(
                                tb['Trade_Type'].isin(['short', 'Add_Short']),  # For Short and Add_Short
                                tb['Entry_Price'] - tb['Exit_Price'],
                                0  # Default case if Trade_Type is something unexpected
                            )
                        )
                        
                        tb['Slippage'] = SLIPPAGE_ * (tb['Entry_Price'] + tb['Exit_Price'])
                        tb['PnL w cs'] = tb['PnL'] - tb['Slippage']
                        tb['Qty'] = abs(RPT_ / 100 * portfolio / (tb['Entry_Price'] - tb['Initial SL']))
                        tb['Profit_Loss INR'] = tb['Qty'] * tb['PnL w cs']
                        tb['ROI%'] = tb['Profit_Loss INR'] * 100 / portfolio
                        tb['Entry_Date'] = pd.to_datetime(tb['Entry_Date'])
                        tb['Trade Year'] = tb['Entry_Date'].dt.year
                        # tb_long_only = tb[(tb['Trade_Type'] == 'Long') | (tb['Trade_Type'] == 'Add_Long')].copy()
                        # tb_long_only['Cumulative_PnL'] = tb_long_only['PnL w cs'].cumsum()
                        # tb_short_only = tb[(tb['Trade_Type'] == 'Short') | (tb['Trade_Type'] == 'Add_Short')].copy()
                        # tb_short_only['Cumulative_PnL'] = tb_short_only['PnL w cs'].cumsum()
                        
                        stats1 = generate_stats(tb, variation)
                        for x, y in stats1.items():
                            if x > 4:
                                # print('Long Only')
                                print(pd.DataFrame(y).to_string())
                                stats_dictionary[x] = y
                            break

EMA1:8, EMA2:8 X:4, Y:4, PCT:0.05


TypeError: calculate_signals() takes 5 positional arguments but 6 were given